<h1>Instructions</h1>
<ol>
<li>Click the button "Not Trusted" on the menu bar of this notebook (at the top-right), and change the value to "Trusted". 
<li>Click Cell -> Run All. If you skip this step you might get the error "Cell magic `%%sql` not found."
<li>In the cell below each question, write a SQL query for the question.  
<li>After you enter a query, press Shift + Enter to run the cell. 
<li>After execution, the system will output the query result and say "CORRECT" if the query works for the sample dataset. Otherwise, it will say "INCORRECT" and also display the expected result for your comparison.
<li>Your submission will be tested over a dataset different from and larger than the sample. You will receive full credit for a question if the query returns correct results on the test dataset. 
</ol>

<h2>Notes</h2>
<ul>
<li>The DBMS running on the server is PostgreSQL 10.6. You may write any query to solve the questions as long as it conforms to its syntax. 
<li>You are encouraged to experiment with different queries by creating new code cells with the %%sql tag. However, when you submit your notebook, please make sure that there is exactly one cell with "%%sql n", for $1 \le n \le 10$, so that the autograder works properly. 
<li>Do not modify the line with %%sql; without it, the cell contents will be treated as Python code. 
</ul>

<h2>Example</h2>

As an example, you can copy and paste the following SQL code into the input box for one of the questions, and then press Shift + Enter. 

If the notebook is working properly, then you should see a table of all the bars in the database. If not, please contact the TA's or make a post on Piazza and we will do our best to resolve the issue. 

<h1>Database Schema</h1>

Drinkers(<u>name</u>, <u>addr</u>, hobby, frequent)

Bars(<u>name</u>, addr, owner)

Beers(<u>name</u>, brewer, alcohol)

Drinks(<u>drinkerName</u>, <u>drinkerAddr</u>, <u>beerName</u>, rating)

Sells(<u>barName</u>, <u>beerName</u>, price, discount)

Favorites(<u>drinkerName</u>, <u>drinkerAddr</u>, <u>barName</u>, beerName, season)

In [45]:
from IPython.core.magic import  (
    Magics, magics_class, cell_magic, line_magic
)
from IPython.display import clear_output, display, Markdown

import IPython.core.display as dis
import requests, json
import urllib.parse

server_url = "http://13.58.198.9:8844"

def execQueryEval(query, query_id):
    cmd = server_url + "/test?query~"+urllib.parse.quote(query)+"&nb~"+str(query_id)
    cmd.encode("utf-8")
    r = requests.get(cmd)
    result = json.loads(r.text)
    if result['success'] == 0:
        print("SQL error, please check your SQL clause.")
        return ""
    elif result['correct'] == 0:
        print("INCORRECT")
        print(result['result'])
        return ""
    else:
        print("CORRECT")
        return ""
    return ""

def execQueryResult(query):
    cmd =  server_url + "/result?query~"+urllib.parse.quote(query)
    cmd.encode("utf-8")
    r = requests.get(cmd)
    result = json.loads(r.text)
    if result['success'] == 0:
        print("SQL error, please check your SQL clause.")
        return ""
    else:
        print(result['result'])
        return ""
    return ""

def displayAsMarkdown(text):
    display(Markdown(text))

@magics_class
class SQL(Magics):

    @cell_magic
    def sql(self, params, cell):
        clear_output()
        
        # remove trailing semicolon
        if cell.endswith(';'):
            cell = cell.rstrip(';')
            
        # get the question number
        n = -1
        if len(params.strip()) > 0:
            try:
                n = int(params.strip())
            except Exception:
                pass
        
        # evaluate the query
        if len(cell.strip()) > 1:
            print ("Query output: ")
            displayAsMarkdown(execQueryResult(cell))
            if n > -1:
                print ("Result: ")
                displayAsMarkdown(execQueryEval(cell, n))

## use ipython load_ext mechanism here if distributed
get_ipython().register_magics(SQL)

# hide this code cell
html = """
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input:eq(0)').hide();
    } else {
      $('div.input:eq(0)').show();
    }
    code_shown = !code_shown;
  }
  
  code_shown=true;
  code_toggle();
</script>
"""
dis.display_html(html, raw=True)

<h2>Question 1:</h2><br>
In California it is illegal to sell beverages with >60% alcohol content. Find the beers in the database that are illegal to sell in California. <br>

<b>Note</b> - alcohol concentration is represented as a floating point number between 0 and 1.

In [46]:
%%sql 1

SELECT name
FROM Beers
WHERE alcohol > 0.6

Query output: 
[['Bud Super'], ['Rum'], ['Vodka']]


Result: 
CORRECT


In [ ]:
%%sql 1


<h2>Question 2:</h2><br>
Find the name and location of bars that sell beers made by the 'Boston Beer Co'.

In [47]:
%%sql 2

SELECT Bars.name,Bars.addr
FROM Bars
WHERE Bars.name IN (
    SELECT Sells.barName
    FROM Sells, Beers
    WHERE Sells.beerName = Beers.name
        and Beers.brewer = 'Boston Beer Co'
) 

Query output: 
[['Green Bar', '100 Green St'], ['Murphy Bar', '604 Green St'], ['Purple Bar', '404 Purple St'], ['Red Lion Bar', '211 Green St'], ['Sober Bar', '102 Purple St']]


Result: 
CORRECT


<h2>Question 3:</h2><br>
Find all of the different beers that are sold in bars located on 'Green St'. <br>

<b>Note</b> - you can assume that address is always in the format '[number] [street]' <br>


In [48]:
%%sql 3

SELECT DISTINCT beerName
FROM Sells
WHERE  barName IN(
    SELECT Bars.name 
    FROM Bars
    WHERE Bars.addr LIKE '%Green St%'    
)



Query output: 
[['Bud'], ['Bud Lite'], ['Bud Super'], ['Coors'], ['Rum'], ['Sam Adams'], ['Vodka']]


Result: 
CORRECT


<h2>Question 4:</h2><br>
Find pairs of drinkers (name1, addr1, name2, addr2) who share a favorite drink and who frequent the same bar.<br>

<b>Note</b> - the context of each person's favorite drink (barName and season) do not need to be the same.<br>

<b>Note</b> - it is expected that name1 || addr1 > name2 || addr2, so that the same information does not appear twice.

In [49]:
%%sql 4

(SELECT DISTINCT D1.name,D1.addr,D2.name,D2.addr
FROM Drinkers D1, Drinkers D2
WHERE D1.frequent = D2.frequent
    and D1.name > D2.name)
INTERSECT
(SELECT DISTINCT F1.drinkerName,F1.drinkerAddr,F2.drinkerName,F2.drinkerAddr
FROM Favorites F1,Favorites F2
WHERE F1.beerName = F2.beerName
    and F1.drinkerName > F2.drinkerName
)


Query output: 
[['Gunter', 'Green St', 'Frank', 'Green St'], ['Helen', 'First St', 'Frank', 'Green St'], ['Helen', 'First St', 'Gunter', 'Green St']]


Result: 
CORRECT


<h2>Question 5:</h2><br>
Find the undiscounted price at which each bar sells the 'Sam Adams' beer

<b>Note</b> - the Sells.price attribute describes the price <b>after</b> all discounts are applied.<br>

<b>Note</b> - the discount attribute is a number between 0 and 1, such that discountedPrice = undiscountedPrice * discount

In [50]:
%%sql 5

SELECT Sells.barName,Sells.price/Sells.discount
FROM Sells
WHERE Sells.beerName = 'Sam Adams'


Query output: 
[['Green Bar', 22.5], ['Murphy Bar', 13.3333], ['Purple Bar', 16.6667], ['Red Lion Bar', 10.0], ['Sober Bar', 60.0]]


Result: 
CORRECT


<h2>Question 6:</h2><br>
Find the bars that sell a unique beer (a beer that no other bars have)

In [51]:
%%sql 6
SELECT barName
FROM Sells
WHERE  beerName IN(
    SELECT beerName
    FROM Sells 
    GROUP BY beerName
    HAVING COUNT(barName)=1
)

Query output: 
[['Green Bar']]


Result: 
CORRECT


<h2>Question 7:</h2><br>
Find the bars that serve the Sam Adams beer cheaper than the average market price.

<b>Note</b> - the average market price is defined as the average price of the beer across all bars that sell the beer. 

In [52]:
%%sql 7
SELECT barName
FROM sells 
WHERE beerName = 'Sam Adams'
    and price < ( 
SELECT AVG(price) 
FROM Sells
WHERE beerName = 'Sam Adams'
GROUP BY beerName
    )


Query output: 
[['Murphy Bar'], ['Red Lion Bar']]


Result: 
CORRECT


<h2>Question 8:</h2><br>
Find pairs of bars (Name1, Name2) that sell the same set of drinks. <br>

<b>Note</b> - for each pair, we would like for Name1 > Name2, so that the same information does not appear twice.

In [53]:
%%sql 8
(
SELECT pair_count.Bar1, pair_count.Bar2
FROM
(
    SELECT (same_beer.bar1) AS Bar1,(same_beer.bar2) AS Bar2, COUNT(same_beer) AS p_count
    FROM
    (
        SELECT (S1.barName) AS bar1,(S2.barName) AS bar2,(S1.barName, S2.barName) AS pair
        FROM Sells S1, Sells S2
        WHERE S1.beerName = S2.beerName
            and S1.barName > S2.barName
    )AS same_beer
    GROUP BY same_beer.bar1, same_beer.bar2
) AS pair_count,
(
    SELECT (bar_counts.barName) AS barname, (bar_counts.counts) AS b_count
    FROM
    (
        SELECT barName, COUNT(beerName)AS counts
        FROM Sells
        GROUP BY barName
    )AS bar_counts
) AS barName_count
WHERE pair_count.Bar1 = barName_count.barname 
    and pair_count.p_count = barName_count.b_count
)
INTERSECT
(
SELECT pair_count.Bar1, pair_count.Bar2
FROM
(
    SELECT (same_beer.bar1) AS Bar1,(same_beer.bar2) AS Bar2, COUNT(same_beer) AS p_count
    FROM
    (
        SELECT (S1.barName) AS bar1,(S2.barName) AS bar2,(S1.barName, S2.barName) AS pair
        FROM Sells S1, Sells S2
        WHERE S1.beerName = S2.beerName
            and S1.barName > S2.barName
    )AS same_beer
    GROUP BY same_beer.bar1, same_beer.bar2
) AS pair_count,
(
    SELECT (bar_counts.barName) AS barname, (bar_counts.counts) AS b_count
    FROM
    (
        SELECT barName, COUNT(beerName)AS counts
        FROM Sells
        GROUP BY barName
    )AS bar_counts
) AS barName_count
WHERE pair_count.Bar2 = barName_count.barname 
    and pair_count.p_count = barName_count.b_count
)


Query output: 
[['Red Lion Bar', 'Murphy Bar']]


Result: 
CORRECT


<h2>Question 9:</h2><br>
Suppose we are interested in computing statistics about the ratings of people's favorite beers. Obtain a list of all ratings for all beers that have been favorited by >= 3 different people. 

In [54]:
%%sql 9
SELECT rating
FROM Drinks
WHERE beerName IN (
    SELECT beerName
    FROM Favorites
    GROUP BY beerName
    HAVING COUNT(DISTINCT drinkerName)>=3
)

Query output: 
[[2], [2], [4], [5]]


Result: 
CORRECT


<h2>Question 10:</h2><br>
Find the most expensive beer, along with its price, that is sold at each bar. <br>

<b>Note</b> - the expected output format is (bar name, name of most expensive beer at that bar, price of most expensive beer) <br>

<b>Note</b> - you can assume that there will be at most one row for each bar.

In [55]:
%%sql 10

SELECT Sells.barName, Sells.beerName, New.max_price
FROM
(
    SELECT barName,MAX(price) AS max_price
    FROM Sells
    GROUP BY barName
)AS New, Sells
WHERE New.barName=Sells.barName
    and Sells.price=New.max_price

Query output: 
[['Green Bar', 'Sam Adams', 4.5], ['Murphy Bar', 'Rum', 15.0], ['Purple Bar', 'Sam Adams', 5.0], ['Red Lion Bar', 'Vodka', 15.0], ['Sober Bar', 'Sam Adams', 6.0]]


Result: 
CORRECT
